In [1]:
from openai import OpenAI
from pydantic import BaseModel
import json

In [2]:
import pandas as pd

In [3]:
from llmdomainrating import *

In [4]:
openai_api_key = load_api_key("../../api_keys.json", 'OpenAI')

In [5]:
DomainRating.model_json_schema()

{'properties': {'rating': {'description': 'Domain credibility rating, ranging from -1 to 1',
   'maximum': 1,
   'minimum': -1,
   'title': 'Rating',
   'type': 'number'},
  'domain': {'description': 'The domain name.',
   'title': 'Domain',
   'type': 'string'},
  'explanation': {'description': 'Explanation of the rating.',
   'title': 'Explanation',
   'type': 'string'}},
 'required': ['rating', 'domain', 'explanation'],
 'title': 'DomainRating',
 'type': 'object'}

In [58]:
json.dumps(DOMAIN_RATING_JSON_SCHEMA)

'{"properties": {"rating": {"description": "Domain credibility rating, ranging from -1 to 1", "title": "Rating", "type": "number"}, "domain": {"description": "The domain name", "title": "Domain", "type": "string"}, "explanation": {"description": "Explanation of the rating.", "title": "Explanation", "type": "string"}}, "additionalProperties": false, "required": ["rating", "domain", "explanation"], "title": "DomainRating", "type": "object"}'

In [7]:
client = OpenAI(api_key=openai_api_key)

In [7]:
SYS_BASE

'You are an assistant to determine the credibility of websites.'

In [8]:
USER_INSTRUCTION

"\n    Rate the website's credibility: {domain}, on a scale between 0 and 1, where 0 means very low credibility and 1 means very high credibility.\n    The assistant returns a rating of -1 when the assistant has no knowledge of the website; otherwise, the assistant should provide the best estimation.\n    In addition to the rating, the assistant should also provide a short explanation.\n"

In [9]:
resp = client.responses.parse(
    model='gpt-4o-mini',
    instructions=SYS_BASE,
    input=USER_INSTRUCTION.format(domain="tampabay.com"),
    temperature=0,
    text={
        "format": {
            "type": "json_schema",
            "name": "domain_rating",
            "strict": True,
            "schema": DOMAIN_RATING_JSON_SCHEMA
        }
    }
)

In [23]:
resp.output[0].content[0].text

'{"rating":0.8,"domain":"tampabay.com","explanation":"Tampa Bay Times is a well-established news organization known for its local coverage and investigative journalism. It has a strong reputation in the community and is generally considered a credible source of information."}'

In [24]:
resp.output_text

'{"rating":0.8,"domain":"tampabay.com","explanation":"Tampa Bay Times is a well-established news organization known for its local coverage and investigative journalism. It has a strong reputation in the community and is generally considered a credible source of information."}'

In [39]:
resp.model_dump_json()

'{"id":"resp_681f6b7f561c8190affe32e8fc5e97850a79d0e3bd3bd31e","created_at":1746889599.0,"error":null,"incomplete_details":null,"instructions":"You are an assistant to determine the credibility of websites.","metadata":{},"model":"gpt-4o-mini-2024-07-18","object":"response","output":[{"id":"msg_681f6b86aba0819085340f9bde9d96ac0a79d0e3bd3bd31e","content":[{"annotations":[],"text":"{\\"rating\\":0.8,\\"domain\\":\\"tampabay.com\\",\\"explanation\\":\\"Tampa Bay Times is a well-established news organization known for its local coverage and investigative journalism. It has a strong reputation in the community and is generally considered a credible source of information.\\"}","type":"output_text","parsed":null}],"role":"assistant","status":"completed","type":"message"}],"parallel_tool_calls":true,"temperature":0.0,"tool_choice":"auto","tools":[],"top_p":1.0,"max_output_tokens":null,"previous_response_id":null,"reasoning":{"effort":null,"generate_summary":null,"summary":null},"service_tier":

In [40]:
with open("/Users/yangkc/working/llm/llm_domain_classification/backend/scripts/openai/tampabay.json") as f:
    temp_resp = json.load(f)

In [52]:
DomainRating.model_validate_json(temp_resp['output'][0]['content'][0]['text'])

DomainRating(rating=0.8, domain='tampabay.com', explanation='Tampa Bay Times is a well-established news organization known for its local journalism in the Tampa Bay area. It has a reputation for credible reporting and is a trusted source for news, which contributes to its high credibility rating.')

In [17]:
gpt41nano_df = pd.read_parquet("/Users/yangkc/working/llm/llm_domain_classification/data/intermediate/parsed_responses/openai/gpt-4.1-nano-2025-04-14.parquet")
gpt41mini_df = pd.read_parquet("/Users/yangkc/working/llm/llm_domain_classification/data/intermediate/parsed_responses/openai/gpt-4.1-mini-2025-04-14.parquet")

In [18]:
gpt41mini_df[gpt41mini_df.rating < 0]

,rating,domain,explanation,provider,model
20,-1.0,thebl.tv,There is no available information or credible ...,openai,gpt-4.1-mini-2025-04-14
38,-1.0,newspress.com,I have no information about the credibility of...,openai,gpt-4.1-mini-2025-04-14
89,-1.0,sooeveningnews.com,There is no available information or known rep...,openai,gpt-4.1-mini-2025-04-14
101,-1.0,spartanavenue.com,I have no information or knowledge about the w...,openai,gpt-4.1-mini-2025-04-14
117,-1.0,thejnotes.com,I have no information or knowledge about the w...,openai,gpt-4.1-mini-2025-04-14
...,...,...,...,...,...
5490,-1.0,godanriver.com,There is no available information or known rep...,openai,gpt-4.1-mini-2025-04-14
5544,-1.0,en-volve.com,I have no information or knowledge about the w...,openai,gpt-4.1-mini-2025-04-14
5559,-1.0,pqbnews.com,I have no knowledge or information about the w...,openai,gpt-4.1-mini-2025-04-14
5630,-1.0,mkecitywire.com,There is no available information or known rep...,openai,gpt-4.1-mini-2025-04-14


In [19]:
len(temp_df[temp_df.rating < 0])

34

In [8]:
resp = client.responses.parse(
    model='o3-mini',
    instructions=SYS_BASE,
    input=USER_INSTRUCTION.format(domain="tampabay.com"),
    reasoning={
        "effort": "medium",
        "summary": "auto",
    },
    text={
        "format": {
            "type": "json_schema",
            "name": "domain_rating",
            "strict": True,
            "schema": DOMAIN_RATING_JSON_SCHEMA
        }
    }
)

In [13]:
resp.model_dump_json()

[ResponseReasoningItem(id='rs_681fc002a7dc81909a18da926eb1ad49053bdec52c1d7993', summary=[Summary(text='**Assessing credibility rating**\n\nI’m evaluating the credibility of “tampabay.com,” which is associated with the Tampa Bay Times. Based on its status as a major regional newspaper, it seems credible. The rating scale goes from 0 to 1, and it’s a well-known source. Although some might critique it for political bias, it generally holds a good journalistic reputation. I’d assign a credibility rating of 0.8, reflecting its evidence-based reporting and overall reliability as a regional newspaper.', type='summary_text'), Summary(text="**Finalizing website credibility**\n\nI’m reflecting on the credibility of tampabay.com, which belongs to the Tampa Bay Times. This newspaper is well-regarded in Florida and has even won Pulitzer Prizes, so it's considered a trustworthy source. Given this reputation, I think a rating of 0.8 is appropriate. I’m planning to present this in a structured format

In [4]:
my_client = create_api_client('openai')

In [6]:
my_resp = my_client.query_model("foxnews.com", "o4-mini")

In [8]:
json.loads(my_resp)

{'id': 'resp_6820c3953c4c819184cbffd3fe1bbf770b04470c31819a5b',
 'created_at': 1746977685.0,
 'error': None,
 'incomplete_details': None,
 'instructions': 'You are an assistant to determine the credibility of websites.',
 'metadata': {},
 'model': 'o4-mini-2025-04-16',
 'object': 'response',
 'output': [{'id': 'rs_6820c39923408191bfec04c5ad9939180b04470c31819a5b',
   'summary': [],
   'type': 'reasoning',
   'encrypted_content': None,
   'status': None},
  {'id': 'msg_6820c39c73648191850fe1403da8f9c60b04470c31819a5b',
   'content': [{'annotations': [],
     'text': '{"rating":0.6,"domain":"foxnews.com","explanation":"Fox News is a major, well‐established news outlet with professional reporting and high audience reach, but it has a pronounced conservative editorial slant and has faced criticism for occasional inaccuracies and partisan framing."}',
     'type': 'output_text',
     'parsed': None}],
   'role': 'assistant',
   'status': 'completed',
   'type': 'message'}],
 'parallel_tool_

In [12]:
with open("/Users/yangkc/working/llm/llm_domain_classification/data/intermediate/raw_responses/openai/o1-2024-12-17/thesaturdaypaper.com.au.txt") as f:
    reasoning_output_df = json.load(f)

In [24]:
for item in reasoning_output_df['output']:
    print(item)

{'id': 'rs_682062549fb0819185e6f6c433ea6ec206033a70c5e2b9a8', 'summary': [{'text': '**Rating website credibility**\n\nThe user wants a credibility rating for "thesaturdaypaper.com.au" on a scale from 0 to 1. Based on my understanding, The Saturday Paper is an established Australian weekly newspaper focusing on politics and society. It’s published by Schwartz Media, which is known for reputable publications like The Monthly and The Quarterly Essay. Given its recognition in Australia as a credible news source, I’d assign it a high credibility rating, likely around 0.8 or 0.9.', 'type': 'summary_text'}, {'text': '**Rating The Saturday Paper**\n\nThe user is asking me to rate "thesaturdaypaper.com.au" on a credibility scale from 0 to 1. I know that The Saturday Paper is an established Australian publication focusing on in-depth journalism and commentary on societal and political issues. It’s recognized for its investigative work and good editorial standards, so I’d rate it around 0.8 or 0.

In [25]:
reasoning_output_df['output'][1]['type']

'message'

In [21]:
reasoning_output_df['output'][1]['content'][0]['text']

'Rating: 0.85  \nExplanation: The Saturday Paper, published by Schwartz Media in Australia, is generally regarded as a reputable source for in-depth journalism and analysis, reflecting consistent editorial standards and a track record of credible reporting.'

In [11]:
reasoning_output_df['output']

[{'id': 'rs_682075bcf6688191a4082485a534907f068914144399ed16',
  'summary': [],
  'type': 'reasoning',
  'encrypted_content': None,
  'status': None},
 {'id': 'msg_682075be589c8191ae3e39ed2bf62cde068914144399ed16',
  'content': [{'annotations': [],
    'text': 'Rating: -1\n\nExplanation: I couldn’t find sufficient reliable information or notable references regarding kion546.com. Without verifiable data or recognized reviews, I can’t confidently assess its credibility.',
    'type': 'output_text',
    'parsed': None}],
  'role': 'assistant',
  'status': 'completed',
  'type': 'message'}]

In [26]:
raw_data_dir = "/Users/yangkc/working/llm/llm_domain_classification/data/intermediate/raw_responses/openai/o3-mini-2025-01-31_raw/"

In [27]:
def load_old_resp_text(file_path):
    with open(file_path, "r") as f:
        resp = json.load(f)
    for item in resp["output"]:
        if item["type"] == "message" and item["role"] == "assistant":
            resp_text = item["content"][0]["text"]
            return resp_text
    return None

In [28]:
resp_text_list = []
for file in os.listdir(raw_data_dir):
    resp_text = load_old_resp_text(
        os.path.join(raw_data_dir, file)
    )
    resp_text_list.append(resp_text)

In [48]:
temp_raw_resp_text = resp_text_list[0]

In [49]:
temp_raw_resp_text

'I would assign frankspeech.com a credibility rating of about 0.3 on a 0 to 1 scale.\n\nExplanation:\nBased on available assessments and online reporting, the website tends to present content with a pronounced political slant and has been associated with viewpoints and interpretations that many experts and fact‐checkers consider unreliable or biased. Although it may provide some factual content, its overall editorial approach and tendency toward controversial or extremist positions result in a relatively low rating in terms of journalistic or informational credibility. Keep in mind that any such rating is inherently somewhat subjective and may change as more information becomes available or if the website alters its practices.'

In [45]:
client = OpenAI()

In [46]:
class DomainRatingExtracted(BaseModel):
    rating: float = Field(
        description="Domain credibility rating, ranging from -1 to 1",
    )
    explanation: str = Field(description="Explanation of the rating.")

In [47]:
def query_openai(resp_text, client, model="gpt-4.1-mini"):
    resp = client.responses.parse(
        model=model,
        instructions="""
            Extract structured information from the provided text. Specifically, identify the rating, which is a number between -1 and 1, and the accompanying explanation. Present this data as a JSON object.

            # Steps

            1. **Identify the Rating**: Scan the text to locate a numerical value that represents the rating, ensure it's within the range of -1 to 1. The rating typically appears at the beginning of the input.
            2. **Extract Explanation**: Find the part of the text that provides context or reasoning for the rating.
            3. **Compile JSON**: Format the rating and explanation into a JSON object.
            """,
        input=resp_text,
        temperature=0,
        text_format=DomainRatingExtracted,
    )
    return resp

In [50]:
test_resp = query_openai(temp_raw_resp_text, client)

In [60]:
test_resp.model_dump_json()

'{"id":"resp_68214e4cde4c8191839cd9d77433ca0104bf6cfcffbf8ba2","created_at":1747013196.0,"error":null,"incomplete_details":null,"instructions":"\\n            Extract structured information from the provided text. Specifically, identify the rating, which is a number between -1 and 1, and the accompanying explanation. Present this data as a JSON object.\\n\\n            # Steps\\n\\n            1. **Identify the Rating**: Scan the text to locate a numerical value that represents the rating, ensure it\'s within the range of -1 to 1. The rating typically appears at the beginning of the input.\\n            2. **Extract Explanation**: Find the part of the text that provides context or reasoning for the rating.\\n            3. **Compile JSON**: Format the rating and explanation into a JSON object.\\n            ","metadata":{},"model":"gpt-4.1-mini-2025-04-14","object":"response","output":[{"id":"msg_68214e558ce481918417d54d493ed48f04bf6cfcffbf8ba2","content":[{"annotations":[],"text":"{\\

In [58]:
test_resp.output_text

'{"rating":0.3,"explanation":"Based on available assessments and online reporting, the website tends to present content with a pronounced political slant and has been associated with viewpoints and interpretations that many experts and fact‐checkers consider unreliable or biased. Although it may provide some factual content, its overall editorial approach and tendency toward controversial or extremist positions result in a relatively low rating in terms of journalistic or informational credibility. Keep in mind that any such rating is inherently somewhat subjective and may change as more information becomes available or if the website alters its practices."}'

In [61]:
pd.read_parquet("/Users/yangkc/working/llm/llm_domain_classification/data/intermediate/parsed_responses/openai/o3-mini-2025-01-31.parquet")

,rating,domain,explanation,cost,input_token_count,output_token_count,cached_input_token_count,reasoning_token_count,provider,model
0,0.30,frankspeech.com,Based on available assessments and online repo...,0.005755,104,1282,0,1152,openai,o3-mini-2025-01-31
1,-1.00,oaoa.com,I don't have sufficient information or verifie...,0.003120,104,683,0,640,openai,o3-mini-2025-01-31
2,0.70,wtol.com,WTOL.com is the website for a local television...,0.004148,103,917,0,832,openai,o3-mini-2025-01-31
3,0.70,niagara-gazette.com,Niagara-Gazette.com is associated with a longs...,0.002664,106,579,0,512,openai,o3-mini-2025-01-31
4,0.85,wbaltv.com,This website is the online presence of a long-...,0.002407,104,521,0,448,openai,o3-mini-2025-01-31
5,0.10,toprightnews.com,This site has been widely criticized for publi...,0.004146,105,916,0,832,openai,o3-mini-2025-01-31
6,0.85,elle.com,Elle.com is the online presence of a long-esta...,0.002501,102,543,0,448,openai,o3-mini-2025-01-31
7,-1.00,thepewterplank.com,I don’t have enough reliable information or hi...,0.002049,107,439,0,384,openai,o3-mini-2025-01-31
8,0.00,qanon.pub,qanon.pub is associated with the QAnon conspir...,0.003176,103,696,0,640,openai,o3-mini-2025-01-31
9,1.00,cancer.gov,cancer.gov is the official website of the Nati...,0.001195,102,246,0,192,openai,o3-mini-2025-01-31
